In [7]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize

In [21]:
path = "./amazon_reviews_us_Musical_Instruments_v1_00.tsv"

In [22]:
f = open(path , "rt", encoding="utf8")

In [23]:
header = f.readline()
header = header.strip().split("\t")

In [24]:
dataset = []
for line in f:
    fields = line.strip().split("\t")
    d = dict(zip(header, fields))
    d["star_rating"] = int(d["star_rating"])
    d["helpful_votes"] = int(d["helpful_votes"])
    d["total_votes"] = int(d["total_votes"])
    dataset.append(d)

In [25]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '45610553',
 'review_id': 'RMDCHWD0Y5OZ9',
 'product_id': 'B00HH62VB6',
 'product_parent': '618218723',
 'product_title': 'AGPtek® 10 Isolated Output 9V 12V 18V Guitar Pedal Board Power Supply Effect Pedals with Isolated Short Cricuit / Overcurrent Protection',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'N',
 'review_headline': 'Three Stars',
 'review_body': 'Works very good, but induces ALOT of noise.',
 'review_date': '2015-08-31'}

In [26]:
#useful data structures
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

In [27]:
itemNames = {}

In [28]:
for d in dataset:
    user,item = d["customer_id"], d["product_id"]
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d["product_title"]

In [29]:
def Jaccard(s1,s2):
    numer = len(s1.intersection(s2)) # A kesişim B
    denom = len(s1.union(s2)) #A birleşim B
    return numer / denom

In [30]:
# Find he purchased i, iterate all other items, compute their similarity with i, sort all by
# Jaccard return most similar
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [31]:
dataset[2]

{'marketplace': 'US',
 'customer_id': '6111003',
 'review_id': 'RIZR67JKUDBI0',
 'product_id': 'B0006VMBHI',
 'product_parent': '603261968',
 'product_title': 'AudioQuest LP record clean brush',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Three Stars',
 'review_body': 'removes dust. does not clean',
 'review_date': '2015-08-31'}

In [32]:
query = dataset[2]["product_id"]
mostSimilar(query)

[(0.028446389496717725, 'B00006I5SD'),
 (0.01694915254237288, 'B00006I5SB'),
 (0.015065913370998116, 'B000AJR482'),
 (0.014204545454545454, 'B00E7MVP3S'),
 (0.008955223880597015, 'B001255YL2'),
 (0.008849557522123894, 'B003EIRVO8'),
 (0.008333333333333333, 'B0015VEZ22'),
 (0.00821917808219178, 'B00006I5UH'),
 (0.008021390374331552, 'B00008BWM7'),
 (0.007656967840735069, 'B000H2BC4E')]

In [33]:
itemNames[query]

'AudioQuest LP record clean brush'

In [34]:
[itemNames[x[1]] for x in mostSimilar(query)]

['Shure SFG-2 Stylus Tracking Force Gauge',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'ART Pro Audio DJPRE II Phono Turntable Preamplifier',
 'Signstek Blue LCD Backlight Digital Long-Playing LP Turntable Stylus Force Scale Gauge Tester',
 'Audio Technica AT120E/T Standard Mount Phono Cartridge',
 'Technics: 45 Adaptor for Technics 1200 (SFWE010)',
 'GruvGlide GRUVGLIDE DJ Package',
 'STANTON MAGNETICS Record Cleaner Kit',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'Behringer PP400 Ultra Compact Phono Preamplifier']

In [35]:
# WEll this was slow, Now find users who purchased i, iterate all users who purchased i,
# build a candidate of items they bought, for that items compute the smilarity with i, sort return
def mostSimilarFast(i):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]
    

In [36]:
mostSimilarFast(query)

[(0.028446389496717725, 'B00006I5SD'),
 (0.01694915254237288, 'B00006I5SB'),
 (0.015065913370998116, 'B000AJR482'),
 (0.014204545454545454, 'B00E7MVP3S'),
 (0.008955223880597015, 'B001255YL2'),
 (0.008849557522123894, 'B003EIRVO8'),
 (0.008333333333333333, 'B0015VEZ22'),
 (0.00821917808219178, 'B00006I5UH'),
 (0.008021390374331552, 'B00008BWM7'),
 (0.007656967840735069, 'B000H2BC4E')]

In [37]:
# Rating Prediction
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [38]:
for d in dataset:
    user, item = d["customer_id"], d["product_id"]
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [39]:
ratingMean = sum([d["star_rating"] for d in dataset]) / len(dataset)

In [40]:
ratingMean

4.251102772543146

In [41]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d["product_id"]
        if i2 == item : continue
        ratings.append(d["star_rating"])
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2]))
    if (sum(similarities) > 0):
        weigtedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weigtedRatings) / sum(similarities)
    else:
        # User didn't rated any similar items
        return ratingMean

In [42]:
dataset[1]

{'marketplace': 'US',
 'customer_id': '14640079',
 'review_id': 'RZSL0BALIYUNU',
 'product_id': 'B003LRN53I',
 'product_parent': '986692292',
 'product_title': 'Sennheiser HD203 Closed-Back DJ Headphones',
 'product_category': 'Musical Instruments',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Five Stars',
 'review_body': 'Nice headphones at a reasonable price.',
 'review_date': '2015-08-31'}

In [43]:
u,i = dataset[1]["customer_id"], dataset[1]["product_id"]

In [44]:
predictRating(u,i)

5.0

In [45]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [46]:
alwaysPredictMean = [ratingMean for d in dataset]

In [47]:
cfPredictions = [predictRating(d["customer_id"], d["product_id"]) for d in dataset] #Collaborative Filtering

In [48]:
labels = [d["star_rating"] for d in dataset]

In [49]:
MSE(alwaysPredictMean, labels)

1.4796142779564334

In [50]:
MSE(cfPredictions, labels)

1.6146130004291603

In [51]:
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [52]:
alpha = ratingMean #alpha and beta user parameters we'll fit. Setting initial vals. alpha to mean beta to zero

In [53]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [54]:
# Bias only model
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [55]:
# grad desc. expect a single vector theta that we will unpack to alpha and beta
def unpack (theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases= dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [56]:
def cost(theta, labels,lamb):
    unpack(theta)
    predictions = [prediction(d["customer_id"], d["product_id"]) for d in dataset]
    cost = MSE(predictions, labels)
    print ("MSE= " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [57]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in dataset:
        u,i = ["customer_id"], d["product_id"]
        pred = prediction(u,i)
        diff = pred-d["star_rating"]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(theta)

In [58]:
MSE(alwaysPredictMean, labels) #for comparison

1.4796142779564334

In [59]:
# Gradient Descent with LBFGS, NOT WORKING COULD USE TENSORFLOW
# scipy.optimize.fmin_l_bfgs_b(cost, [ratingMean] + [0.0]*(nUsers+nItems),derivative, args =(labels,0.001))

In [60]:
alpha = ratingMean

In [61]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [62]:
userGamma = {}
itemGamma = {}

In [63]:
K = 2 #number of factors like dimentionality of gamma

In [64]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.01 - 0.05 or k in range(K)]

In [65]:
for i in reviewsPerItem:
    itemGamma[i] = [random.random() *  0.01 - 0.05 or k in range(K)]

In [66]:
def unpack2(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [67]:
def inner(x,y):
    return sum([a*b for a,b in zip(x,y)])

In [68]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [69]:
def cost2(theta, labels,lamb):
    unpack(theta)
    predictions = [prediction(d["customer_id"], d["product_id"]) for d in dataset]
    cost = MSE(predictions, labels)
    print ("MSE= " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost